In [ ]:
!pip install datasets==1.18.3
!pip install transformers==4.11.3
!pip install huggingface_hub==0.1
!pip install torchaudio
!pip install librosa
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 311 kB 7.1 MB/s 
     |████████████████████████████████| 1.1 MB 47.8 MB/s 
     |████████████████████████████████| 140 kB 42.8 MB/s 
     |████████████████████████████████| 212 kB 65.2 MB/s 
     |████████████████████████████████| 101 kB 12.6 MB/s 
     |████████████████████████████████| 596 kB 49.7 MB/s 
     |████████████████████████████████| 144 kB 56.1 MB/s 
     |████████████████████████████████| 271 kB 41.6 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 7.3 MB/s 
     |████████████████████████████████| 880 kB 54.9 MB/s 
     |█████████████

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-crendential store but this isn't the helper defined on your machine.
You will have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal to set it as the default

git config --global credential.helper store


In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


#### Mounting google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset, load_metric, Audio

train = load_dataset("bengaliAI/CommonVoiceBangla", "bn", delimiter='\t', split = 'train').to_pandas()
val = load_dataset("bengaliAI/CommonVoiceBangla", "bn", delimiter='\t', split = 'validation').to_pandas()
test = load_dataset("bengaliAI/CommonVoiceBangla", "bn", delimiter='\t', split = 'test').to_pandas()

Using custom data configuration bengaliAI--CommonVoiceBangla-5bf33f9e391cfca7
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/bengaliAI--CommonVoiceBangla-5bf33f9e391cfca7/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)
Using custom data configuration bengaliAI--CommonVoiceBangla-5bf33f9e391cfca7
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/bengaliAI--CommonVoiceBangla-5bf33f9e391cfca7/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)
Using custom data configuration bengaliAI--CommonVoiceBangla-5bf33f9e391cfca7
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/bengaliAI--CommonVoiceBangla-5bf33f9e391cfca7/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


#### Cleaning & normalization

In [ ]:
!pip install bnunicodenormalizer
!pip install indicparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Viewing the columns

In [ ]:
train.head()

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment,sentence_id,reason
0,6c274d3678353b62f5d3465c771e4a8c5cd917186a5323...,common_voice_bn_30991326.mp3,বাবা সত্যেন ঘোষ।,1.0,1.0,None,None,None,bn,NaN,NaN,NaN
1,3991c2cfe1be64e77f2f6bdaeaae601219af906c1295cd...,common_voice_bn_30991371.mp3,দেওয়ান ফরিদ গাজী বাংলাদেশ আওয়ামী লীগের উপদেষ...,1.0,0.0,None,None,None,bn,NaN,NaN,NaN
2,3991c2cfe1be64e77f2f6bdaeaae601219af906c1295cd...,common_voice_bn_30991410.mp3,এর মাধ্যমে কারও গান গাওয়া দক্ষতা বা নিজস্ব ধর...,0.0,0.0,None,None,None,bn,NaN,NaN,NaN
3,08cb638755d8adbe55e3149211af88853e0d77785397f9...,common_voice_bn_30991432.mp3,"আপনি খুব একটা কথা বলার লোক নন, তাই না?",0.0,0.0,None,None,None,bn,NaN,NaN,NaN
4,51215b6882d5fdc8a9856b095793244085341470a888f5...,common_voice_bn_30991478.mp3,"আপনি খুব একটা কথা বলার লোক নন, তাই না?",0.0,0.0,None,None,চট্টগ্রামের ভাষার সামান্য টানসহ মোটামুটি প্রমি...,bn,NaN,NaN,NaN


#### Keeping only the texts for training, it will convert the dataframe into a pandas series

#### Normalizer function

In [ ]:
from bnunicodenormalizer import Normalizer 
from pprint import pprint
from tqdm import tqdm

def normalizer(df):
    # initialize
    result =[]
    results =[]

    bnorm=Normalizer()
    # normalize

    for i in tqdm(df.index):
        for word in df['sentence'][i].split():
            if bnorm(word)['normalized'] is not None:
                result.append(bnorm(word)['normalized'])
                
        result = " ".join(result)
        results.append(result)
        result = []

    df['normalized'] = results
    return df

In [ ]:
train = normalizer(train)['normalized']
val = normalizer(val)['normalized']
test = normalizer(test)['normalized']

100%|██████████| 7747/7747 [01:06<00:00, 116.73it/s]


In [ ]:
train.to_csv('/content/drive/MyDrive/Commonvoice/New/Training/norm_train.csv', encoding = 'utf-8', index = False)
val.to_csv('/content/drive/MyDrive/Commonvoice/New/Training/norm_val.csv', encoding = 'utf-8', index = False)
test.to_csv('/content/drive/MyDrive/Commonvoice/New/Training/norm_test.csv', encoding = 'utf-8', index = False)

#### Now, let's remove the special characters, because those don't make much big of a sense in terms of speech

In [ ]:
train

0                                          বাবা সত্যেন ঘোষ।
1         দেওয়ান ফরিদ গাজী বাংলাদেশ আওয়ামী লীগের উপদেষ্ট...
2         এর মাধ্যমে কারও গান গাওয়া দক্ষতা বা নিজস্ব ধর্...
3                    আপনি খুব একটা কথা বলার লোক নন, তাই না?
4                    আপনি খুব একটা কথা বলার লোক নন, তাই না?
                                ...                        
206945    রাণীগঞ্জ মূলত এখানকার কয়লা খনি এবং খনিজ সংক্রা...
206946    ওহাইও নদীর মুখোমুখি অবস্থিত, কারখানাটি লম্বা এ...
206947    উল্লেখ্য যে, আইন অমান্য আন্দোলনে বাংলার মুসলমা...
206948    তিনি রাজস্থানের প্রতাপগড় জেলা থেকে রাজস্থান বি...
206949    ইয়েল ইউনিভার্সিটি, অন্য সংবাদপত্রের সাথে যোগায...
Name: normalized, Length: 206950, dtype: object

In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\।\=\—]'

def remove_special_characters(batch):
    batch = re.sub(chars_to_remove_regex, '', batch)
    return batch

In [ ]:
train = train.map(remove_special_characters)
val = val.map(remove_special_characters)
test = test.map(remove_special_characters)

#### Sanity checking of the character removal function

In [ ]:
train

0                                           বাবা সত্যেন ঘোষ
1         দেওয়ান ফরিদ গাজী বাংলাদেশ আওয়ামী লীগের উপদেষ্ট...
2         এর মাধ্যমে কারও গান গাওয়া দক্ষতা বা নিজস্ব ধর্...
3                      আপনি খুব একটা কথা বলার লোক নন তাই না
4                      আপনি খুব একটা কথা বলার লোক নন তাই না
                                ...                        
206945    রাণীগঞ্জ মূলত এখানকার কয়লা খনি এবং খনিজ সংক্রা...
206946    ওহাইও নদীর মুখোমুখি অবস্থিত কারখানাটি লম্বা এব...
206947    উল্লেখ্য যে আইন অমান্য আন্দোলনে বাংলার মুসলমান...
206948    তিনি রাজস্থানের প্রতাপগড় জেলা থেকে রাজস্থান বি...
206949    ইয়েল ইউনিভার্সিটি অন্য সংবাদপত্রের সাথে যোগাযো...
Name: normalized, Length: 206950, dtype: object

#### Now, let's extract all the characters from all the sentences and create the vocabulary. Later we will take a separate approach and extract the graphemes from here and take union of those with the most common graphemes in Bengali to create our vocabulary.

In [ ]:
def remove_spaces(batch):
    batch = re.sub(" ", "", batch)
    return batch

def extract_all_chars(df):
    charslist = "".join(df.to_list())
    charslist = list(set(df))
    return charslist

def create_vocab(dftrain, dfval, dftest):
    trainlist = extract_all_chars(dftrain.map(remove_spaces))
    vallist = extract_all_chars(dfval.map(remove_spaces))
    testlist = extract_all_chars(dftest.map(remove_spaces))
    vocablist = set("".join(trainlist + vallist + testlist))
    vocab = {v: k for k, v in enumerate(sorted(vocablist))}
    return vocab

In [ ]:
vocab_dict = create_vocab(train, val, test)

# Adding word delimieter, unknown and padding tokens:

vocab_dict["|"] = len(vocab_dict)
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

vocab_dict

{'[PAD]': 63,
 '[UNK]': 62,
 '|': 61,
 'ঁ': 0,
 'ং': 1,
 'ঃ': 2,
 'অ': 3,
 'আ': 4,
 'ই': 5,
 'ঈ': 6,
 'উ': 7,
 'ঊ': 8,
 'ঋ': 9,
 'এ': 10,
 'ঐ': 11,
 'ও': 12,
 'ঔ': 13,
 'ক': 14,
 'খ': 15,
 'গ': 16,
 'ঘ': 17,
 'ঙ': 18,
 'চ': 19,
 'ছ': 20,
 'জ': 21,
 'ঝ': 22,
 'ঞ': 23,
 'ট': 24,
 'ঠ': 25,
 'ড': 26,
 'ঢ': 27,
 'ণ': 28,
 'ত': 29,
 'থ': 30,
 'দ': 31,
 'ধ': 32,
 'ন': 33,
 'প': 34,
 'ফ': 35,
 'ব': 36,
 'ভ': 37,
 'ম': 38,
 'য': 39,
 'র': 40,
 'ল': 41,
 'শ': 42,
 'ষ': 43,
 'স': 44,
 'হ': 45,
 'া': 46,
 'ি': 47,
 'ী': 48,
 'ু': 49,
 'ূ': 50,
 'ৃ': 51,
 'ে': 52,
 'ৈ': 53,
 'ো': 54,
 'ৌ': 55,
 '্': 56,
 'ৎ': 57,
 'ড়': 58,
 'ঢ়': 59,
 'য়': 60}

In [ ]:
import json

with open("/content/drive/MyDrive/Commonvoice/New/Training/vocab_dict.json", 'w') as outfile:
    json.dump(vocab_dict, outfile, ensure_ascii=False)   #ensure_ascii set to False to show the Bangla characters, otherwise it doesn't show

#### Use the json file to load the vocabulary into an instance of the Wav2Vec2CTCTokenizer class


In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/Commonvoice/New/Training/vocab_dict.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
#NOTE: Gotta use the vocab json filepath as the first parameter here

In [ ]:
tokenizer

PreTrainedTokenizer(name_or_path='', vocab_size=64, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'})

In [ ]:
repo_name = "wav2vec2-large-xlsr-bn"
tokenizer.push_to_hub(repo_name)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:718: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:718: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/nakkhatra/wav2vec2-large-xlsr-bn into local empty directory.
To https://huggingface.co/nakkhatra/wav2vec2-large-xlsr-bn
   7206096..19fbed2  main -> main



'https://huggingface.co/nakkhatra/wav2vec2-large-xlsr-bn/commit/19fbed2979d3b27ff205ed703349a9a155443643'

#### Building the feature extractor to extract features from the raw audio. XLS-R was pretrained on 16kHz sample rate. Common voice audio has sample rate of 48kHz. So, this has to be downsampled.

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

#### For improved user-friendliness, the feature extractor and tokenizer are wrapped into a single Wav2Vec2Processor class so that one only needs a model and processor object.

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

## Preprocess the data